In [ ]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [ ]:
FILE_PATH = '/home/prashant/Downloads/brain_w/data/news.csv'
df = pd.read_csv(FILE_PATH)

In [ ]:
df['headline'] = df['headline'].apply(preprocess_data)

In [ ]:
df['text'] = df['text'].apply(preprocess_data)

In [ ]:
df["merge"] = df["headline"].map(str) +" "+ df["text"]

In [ ]:
import plotly.plotly as py

In [ ]:
# from gensim.models import Word2Vec
# # define training data
# sentences = l
# # train model
# model_int = Word2Vec(sentences, min_count=4,size = 300)
# # summarize the loaded model
# print(model_int)
# # summarize vocabulary
# #words = list(model.wv.vocab)
# #print(words)
# # access vector for one word
# #print(model['sentence'])
# # save model
# #model.save('model.bin')
# # load model
# #new_model = Word2Vec.load('model.bin')
# #print(new_model)

In [ ]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

In [ ]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:
                
                get_word_vec = model[w]
            except:
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen

In [ ]:
vectors = get_vector(model,list(df['merge']))

In [ ]:
import time

from sklearn.manifold import TSNE

n_sne = 7000

time_start = time.time()
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(vectors)




In [ ]:
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X= tsne_results  # For reproducibility

range_n_clusters = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,17]


In [ ]:
from sklearn.mixture import GaussianMixture


In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
import hdbscan

In [ ]:
figures = []

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig = tools.make_subplots(rows=1, cols=2,
                              print_grid=False,
                              subplot_titles=('The silhouette plot for the various clusters.',
                                              'The visualization of the clustered data.'))

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    fig['layout']['xaxis1'].update(title='The silhouette coefficient values',
                                   range=[-0.1, 1])
   
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    fig['layout']['yaxis1'].update(title='Cluster label',
                                   showticklabels=False,
                                   range=[0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.

#Predict GMM cluster membership
    gm_messy = GaussianMixture(n_components=n_clusters,\
                               covariance_type='diag',max_iter=100,tol=1e-15,reg_covar =1e-1).fit(X)
    #hdb = hdbscan.HDBSCAN(min_cluster_size=n_clusters)
    #hdb_pred = hdb.fit(X)

    
    cluster_labels = gm_messy.predict(X)

    #silhouette_score(vectors, p)
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

In [ ]:
gm_try = GaussianMixture(n_components=2,\
                           covariance_type='spherical',max_iter=100,tol=1e-15,reg_covar =1e-1).fit(X)
#hdb = hdbscan.HDBSCAN(min_cluster_size=n_clusters)
#hdb_pred = hdb.fit(X)


cluster_labels_try = gm_try.predict(X)
silhouette_avg = silhouette_score(X, cluster_labels_try)
print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    #sample_silhouette_values = silhouette_samples(X, cluster_labels)

In [ ]:
# data_vocab = vectorizer.vocabulary_
# dataset_v = data_vocab.keys()
# print len(dataset_v)


# len(set(vocab).intersection(dataset_v))

In [ ]:
np.savetxt("/home/prashant/faad_clusters.txt", np.matrix(X))

In [ ]:
def elbow_plot(data, maxK=50):
    """
        parameters:
        - data: pandas DataFrame (data to be fitted)
        - maxK (default = 10): integer (maximum number of clusters with which to run k-means)
        
    """
    sse = {}
    for k in range(1, maxK):
        print("k: ", k)
       
        
        kmeans = KMeans(n_clusters=k, max_iter=300, n_init=100, random_state=0).fit(data)
        
        #data = kmeans.labels_
        # Inertia: Sum of distances of samples to their closest cluster center
        sse[k] = kmeans.inertia_
        print(kmeans.inertia_)
    plt.figure()
    plt.plot(list(sse.keys()), list(sse.values()))
    plt.show()
    return sse


In [ ]:
import matplotlib.pyplot as plt
k = elbow_plot(vectors)

In [ ]:
kmeans = KMeans(n_clusters=2, max_iter=300, n_init=1000, random_state=0).fit(vectors)
pred = (kmeans.predict(vectors))
import sklearn
sklearn.metrics.silhouette_score(vectors, pred)

In [ ]:
# >>> from sklearn.cluster import DBSCAN
# dbsc = DBSCAN(eps = .3, min_samples = 15).fit(vectors)
# #sklearn.metrics.silhouette_score(vectors, pred)
# labels = dbsc.labels_

In [ ]:
labels

In [ ]:
from sklearn.mixture import GaussianMixture

#Predict GMM cluster membership
gm_messy = GaussianMixture(n_components=2,covariance_type='spherical',max_iter=100,tol=1e-15,reg_covar =1e-1).fit(vectors)
p = gm_messy.predict(vectors)
p
sklearn.metrics.silhouette_score(vectors, p)

In [ ]:
resu = list(zip(df['id'],cluster_labels_try))

In [ ]:
result_df = pd.DataFrame(resu,columns=['id','cluster'],index=None)

In [ ]:
result_df.to_csv('/home/prashant/faad_result.csv',index=None)

In [ ]:
np.savetxt("/home/prashant/Downloads/brain_w/data/dataset/test/sample_submission/vec.txt", np.matrix(vectors))

In [ ]:
result_df.shape

In [ ]:
x = list(map(lambda x: x[0] ,tsne_results))
y = list(map(lambda x: x[1] ,tsne_results))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(x, y)

In [ ]:
from sklearn.mixture import GaussianMixture

#Predict GMM cluster membership
gm_messy = GaussianMixture(n_components=3).fit(vectors)
gm_messy.predict(vectors)

In [ ]:
gm_messy

In [ ]:
gm_messy.predict(vectors)

In [ ]:
import json

In [ ]:
with open('/home/prashant/output.jsonl') as f:
    data = json.load(f)

#print(data)

In [ ]:
len(data['features'][0]['layers'][0]['values'])

In [ ]:
def data_generator(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            yield json.loads(line)
    return


In [ ]:
dat = data_generator('/home/prashant/output.jsonl')

In [ ]:
sample = dat.next()
sample